In [1]:
from modelzipper.tutils import *

content = auto_read_data("/mnt/petrelfs/tangzecheng/local_data/processed_multi_hop/random_drop_1/infernet_res.pkl")

/mnt/petrelfs/tangzecheng/anaconda3/envs/zecheng_new/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModelZipper is ready for launch🚀 | Current Version🦄 >>> 0.2.7 <<< | AOE Time🕒 2024-12-22 14:17:32
begin to read data from /mnt/petrelfs/tangzecheng/local_data/processed_multi_hop/random_drop_1/infernet_res.pkl | file size: 108.11 MB | file type: pkl


In [2]:
len(content)

6000

In [5]:
content[2]


{'answer': "The purpose of the shared dev server, known as 'dev', is to operate on AWS servers, testing production-like setups against a dev API with limited test data, distinct from Ember's local development environment. It auto-deploys after each successful CI build. The setup involves configuring different Cloudfront and S3 settings for dev and stage environments in `config/environment.js`, and the deployment process varies between production, dev, and stage environments. In contrast, developers face the challenge of documentation when creating a shared component library in Ember. Existing documentation tools for JavaScript like YUIDoc, JSDoc, ESDoc, and Documentation.js have limitations for Ember developers, including lack of support for modern JavaScript features, no tailored support for Ember-specific concepts, and absence of live examples and demos. This has led to a gap in fully documented addons with good examples and well-described APIs. The development of ember-cli-addon-doc

In [8]:
content['qa2']['0k']['datas'][0]

{'question': 'The football is in the kitchen.',
 'input': 'Mary journeyed to the bathroom. Sandra went to the garden. Daniel went back to the garden. Daniel went to the office. Sandra grabbed the milk there. Sandra put down the milk there. Daniel went to the hallway. Sandra got the milk there. Daniel went to the garden. Daniel journeyed to the kitchen. Daniel journeyed to the bedroom. Mary journeyed to the garden. Daniel took the football there. Mary moved to the office. Sandra travelled to the bedroom. Daniel dropped the football. Sandra left the milk there. Daniel grabbed the football there. Sandra grabbed the milk there. Daniel went to the kitchen.',
 'golden': 'yes'}